In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [7]:
train_df = pd.read_csv('/kaggle/input/churn-detection/train.csv')
test_df = pd.read_csv('/kaggle/input/churn-detection/test.csv')

NameError: name 'pd' is not defined

In [ ]:



# Convert 'TotalCharges' to numeric
train_df['TotalCharges'] = pd.to_numeric(train_df['TotalCharges'], errors='coerce')
test_df['TotalCharges'] = pd.to_numeric(test_df['TotalCharges'], errors='coerce')



In [ ]:
# Fill missing values with the median
train_df['TotalCharges'] = train_df['TotalCharges'].fillna(train_df['TotalCharges'].median())
test_df['TotalCharges'] = test_df['TotalCharges'].fillna(test_df['TotalCharges'].median())


In [ ]:
# Drop 'id' column
train_df.drop(columns=['id'], inplace=True)
test_ids = test_df['id']
test_df.drop(columns=['id'], inplace=True)

In [ ]:
# Encode categorical variables
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()
categorical_columns.remove('Churn')

In [ ]:
# Encode using LabelEncoder
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

In [ ]:
# Encode target
target_encoder = LabelEncoder()
train_df['Churn'] = target_encoder.fit_transform(train_df['Churn'])

In [ ]:
train_df.head()


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_df = pd.read_csv('/kaggle/input/churn-detection/train.csv')
test_df = pd.read_csv('/kaggle/input/churn-detection/test.csv')

In [ ]:
train_df['TotalCharges'] = pd.to_numeric(train_df['TotalCharges'], errors='coerce')
test_df['TotalCharges'] = pd.to_numeric(test_df['TotalCharges'], errors='coerce')

In [ ]:
train_df['TotalCharges'] = train_df['TotalCharges'].fillna(train_df['TotalCharges'].median())
test_df['TotalCharges'] = test_df['TotalCharges'].fillna(test_df['TotalCharges'].median())


In [ ]:
train_df.drop(columns=['id','PhoneService','StreamingTV','StreamingMovies','Dependents','SeniorCitizen','InternetService','gender','MultipleLines','DeviceProtection','OnlineBackup','PaperlessBilling','Partner'], inplace=True)
test_ids = test_df['id']
test_df.drop(columns=['id','PhoneService','StreamingTV','StreamingMovies','Dependents','SeniorCitizen','InternetService','gender','MultipleLines','DeviceProtection','OnlineBackup','PaperlessBilling','Partner'], inplace=True)

In [ ]:
#Encode categorical variables
categorical_columns = train_df.select_dtypes(include=['object']).columns.tolist()
categorical_columns.remove('Churn')


In [ ]:
# Encode using LabelEncoder
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

In [ ]:
# Encode target
target_encoder = LabelEncoder()
train_df['Churn'] = target_encoder.fit_transform(train_df['Churn'])


In [ ]:
# Show cleaned and encoded data
train_df.head()
test_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
#Split the training data
X = train_df.drop(columns=['Churn'])
y = train_df['Churn']

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
 #Train a Random Forest Classifier
model = RandomForestClassifier(random_state=100)
model.fit(X, y)

In [ ]:
feature_scores = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

feature_scores

In [ ]:
# Evaluate on validation set
y_pred = model.predict(test_df)
#accuracy = accuracy_score(y_val, y_pred)
#report = classification_report(y_val, y_pred, target_names=target_encoder.classes_)

In [ ]:
accuracy, report

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Refit encoder like you did when cleaning train.csv
target_encoder = LabelEncoder()
target_encoder.fit(['No', 'Yes'])  # very important: correct order!

# Decode predictions
decoded_preds = target_encoder.inverse_transform(y_pred)

# Now generate valid submission
rand_df = pd.read_csv('/kaggle/input/churn-detection/test.csv')
submission_df = pd.DataFrame({'id': rand_df['id'], 'Churn': decoded_preds})
submission_df.to_csv('submission_random_forest.csv', index=False)

In [ ]:
submission_df.to_csv('submission_random_forest.csv', index=False)

In [ ]:
output_df = pd.read_csv('/kaggle/working/submission_random_forest.csv')
output_df

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

In [ ]:
train_df = pd.read_csv('/kaggle/input/churn-detection/train.csv')
test_df = pd.read_csv('/kaggle/input/churn-detection/test.csv')

In [ ]:
train_df['TotalCharges'] = pd.to_numeric(train_df['TotalCharges'], errors='coerce')
test_df['TotalCharges'] = pd.to_numeric(test_df['TotalCharges'], errors='coerce')


In [ ]:
train_df['TotalCharges'] = train_df['TotalCharges'].fillna(train_df['TotalCharges'].median())
test_df['TotalCharges'] = test_df['TotalCharges'].fillna(test_df['TotalCharges'].median())

In [ ]:
test_ids = test_df['id']
train_df.drop(columns=['id'], inplace=True)
test_df.drop(columns=['id'], inplace=True)


In [ ]:
replace_cols = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines'
]
for col in replace_cols:
    train_df[col] = train_df[col].replace({'No internet service': 'No', 'No phone service': 'No'})
    test_df[col] = test_df[col].replace({'No internet service': 'No', 'No phone service': 'No'})

In [ ]:
cat_cols = train_df.select_dtypes(include='object').columns.tolist()
cat_cols.remove('Churn')
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

In [ ]:
target_encoder = LabelEncoder()
train_df['Churn'] = target_encoder.fit_transform(train_df['Churn'])

In [ ]:
X = train_df.drop(columns=['Churn'])
y = train_df['Churn']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train[['MonthlyCharges', 'TotalCharges', 'tenure']] = scaler.fit_transform(X_train[['MonthlyCharges', 'TotalCharges', 'tenure']])
X_val[['MonthlyCharges', 'TotalCharges', 'tenure']] = scaler.transform(X_val[['MonthlyCharges', 'TotalCharges', 'tenure']])
test_df[['MonthlyCharges', 'TotalCharges', 'tenure']] = scaler.transform(test_df[['MonthlyCharges', 'TotalCharges', 'tenure']])

In [ ]:
model = XGBClassifier(
    learning_rate =0.1,
              n_estimators=40,
              max_depth=5,
              min_child_weight=2,
              gamma=0.1,
              subsample=0.85,
              colsample_bytree=0.8,
              objective= 'binary:logistic',
              nthread=4,
              scale_pos_weight=1,
              seed=27)

model.fit(X_train, y_train)

In [ ]:
val_preds = model.predict(X_val)
print("\nValidation Accuracy:", accuracy_score(y_val, val_preds))
print(classification_report(y_val, val_preds, target_names=target_encoder.classes_))

In [ ]:
X[['MonthlyCharges', 'TotalCharges', 'tenure']] = scaler.fit_transform(X[['MonthlyCharges', 'TotalCharges', 'tenure']])
model.fit(X, y)

In [ ]:
preds = model.predict(test_df)
preds_labels = target_encoder.inverse_transform(preds)

submission = pd.DataFrame({
    'id': test_ids,
    'Churn': preds_labels
})
submission.to_csv('submission.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from xgboost import XGBClassifier

In [ ]:
train_df = pd.read_csv('/kaggle/input/churn-detection/train.csv')
test_df = pd.read_csv('/kaggle/input/churn-detection/test.csv')

In [ ]:
train_df['TotalCharges'] = pd.to_numeric(train_df['TotalCharges'], errors='coerce')
test_df['TotalCharges'] = pd.to_numeric(test_df['TotalCharges'], errors='coerce')

In [ ]:
train_df['TotalCharges'] = train_df['TotalCharges'].fillna(train_df['TotalCharges'].median())
test_df['TotalCharges'] = test_df['TotalCharges'].fillna(test_df['TotalCharges'].median())

In [ ]:
test_ids = test_df['id']
train_df.drop(columns=['id'], inplace=True)
test_df.drop(columns=['id'], inplace=True)

In [ ]:
replace_cols = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines'
]
for col in replace_cols:
    train_df[col] = train_df[col].replace({'No internet service': 'No', 'No phone service': 'No'})
    test_df[col] = test_df[col].replace({'No internet service': 'No', 'No phone service': 'No'})

In [ ]:
train_df['TotalSpent'] = train_df['MonthlyCharges'] * train_df['tenure']
test_df['TotalSpent'] = test_df['MonthlyCharges'] * test_df['tenure']

train_df['TenureGroup'] = pd.cut(train_df['tenure'], bins=[0, 12, 24, 48, 60, 72], labels=[1, 2, 3, 4, 5])
test_df['TenureGroup'] = pd.cut(test_df['tenure'], bins=[0, 12, 24, 48, 60, 72], labels=[1, 2, 3, 4, 5])

train_df['TotalServices'] = (train_df[['PhoneService', 'MultipleLines', 'OnlineSecurity',
                                       'OnlineBackup', 'DeviceProtection', 'TechSupport',
                                       'StreamingTV', 'StreamingMovies']] == 'Yes').sum(axis=1)

test_df['TotalServices'] = (test_df[['PhoneService', 'MultipleLines', 'OnlineSecurity',
                                     'OnlineBackup', 'DeviceProtection', 'TechSupport',
                                     'StreamingTV', 'StreamingMovies']] == 'Yes').sum(axis=1)

In [ ]:
cat_cols = train_df.select_dtypes(include='object').columns.tolist()
cat_cols.remove('Churn')
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

In [ ]:
target_encoder = LabelEncoder()
train_df['Churn'] = target_encoder.fit_transform(train_df['Churn'])

In [ ]:
X = train_df.drop(columns=['Churn'])
y = train_df['Churn']

# Scale numeric features
scaler = StandardScaler()
num_cols = ['MonthlyCharges', 'TotalCharges', 'tenure', 'TotalSpent']
X[num_cols] = scaler.fit_transform(X[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Compute class imbalance weight
scale_pos_weight = (y == 0).sum() / (y == 1).sum()

In [ ]:
model = XGBClassifier(
    learning_rate=0.05,
    n_estimators=100,
    max_depth=4,
    min_child_weight=2,
    gamma=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    scale_pos_weight=scale_pos_weight,
    use_label_encoder=False,
    eval_metric='logloss',
    enable_categorical = True,
    random_state=42
)

model.fit(X_train, y_train)

In [ ]:
val_preds = model.predict(X_val)
val_probs = model.predict_proba(X_val)[:, 1]

print("\n🔍 Validation Accuracy:", accuracy_score(y_val, val_preds))
print("🔍 ROC AUC:", roc_auc_score(y_val, val_probs))
print(classification_report(y_val, val_preds, target_names=target_encoder.classes_))

In [ ]:
plt.figure(figsize=(10, 6))
importance = model.feature_importances_
feat_names = X.columns
sorted_idx = np.argsort(importance)

plt.barh(feat_names[sorted_idx], importance[sorted_idx])
plt.title(" Feature Importance")
plt.xlabel("Importance Score")
plt.show()


In [ ]:
X_scaled = scaler.fit_transform(X[num_cols])
X[num_cols] = X_scaled
model.fit(X, y)

In [ ]:
test_preds = model.predict(test_df)
test_labels = target_encoder.inverse_transform(test_preds)

In [ ]:
submission = pd.DataFrame({
    'id': test_ids,
    'Churn': test_labels
})
submission.to_csv('submission2.csv', index=False)
print("\n✅ submission.csv created for Kaggle upload")

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
import optuna
import warnings

In [12]:
train_df = pd.read_csv('/kaggle/input/churn-detection/train.csv')
test_df = pd.read_csv('/kaggle/input/churn-detection/test.csv')

In [13]:
# Store test IDs for submission and drop the ID columns
test_ids = test_df['id']
train_df = train_df.drop('id', axis=1)
test_df = test_df.drop('id', axis=1)

In [14]:
# Convert 'TotalCharges' to numeric, coercing errors to NaN
train_df['TotalCharges'] = pd.to_numeric(train_df['TotalCharges'], errors='coerce')
test_df['TotalCharges'] = pd.to_numeric(test_df['TotalCharges'], errors='coerce')


In [16]:
train_df['TotalCharges'] = train_df['TotalCharges'].fillna(train_df['TotalCharges'].median())
test_df['TotalCharges'] = test_df['TotalCharges'].fillna(test_df['TotalCharges'].median())

In [17]:
# Consolidate 'No internet service' and 'No phone service' for consistency
replace_cols = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines'
]
for col in replace_cols:
    train_df[col] = train_df[col].replace({'No internet service': 'No', 'No phone service': 'No'})
    test_df[col] = test_df[col].replace({'No internet service': 'No', 'No phone service': 'No'})

In [18]:
# Create new features based on existing data
for df in [train_df, test_df]:
    # Ratio of tenure to monthly charges
    df['TenureMonthlyRatio'] = df['tenure'] / (df['MonthlyCharges'] + 1e-6)
    
    # Total charges per month
    df['MonthlyContractValue'] = df['TotalCharges'] / (df['tenure'] + 1)
    
    # Total number of additional services
    additional_services = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
    df['AdditionalServicesCount'] = (df[additional_services] == 'Yes').sum(axis=1)

In [19]:
# ------------------------------------------------------------------
# 3. PREPARE DATA FOR MODELING
# ------------------------------------------------------------------

# Encode the target variable 'Churn'
target_encoder = LabelEncoder()
train_df['Churn'] = target_encoder.fit_transform(train_df['Churn'])


In [20]:

# Separate features (X) and target (y)
X = train_df.drop('Churn', axis=1)
y = train_df['Churn']


In [21]:
# Identify numeric and categorical features
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

In [22]:
# Create preprocessing pipelines for numeric and categorical data
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))])

In [23]:
# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)


In [32]:
# Define the model pipeline ONCE, outside the optimization function
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, n_jobs=-1))
])


In [33]:

# ------------------------------------------------------------------
# 5. HYPERPARAMETER OPTIMIZATION WITH OPTUNA
# ------------------------------------------------------------------

def objective(trial):
    # Define the hyperparameter search space using the 'classifier__' prefix for the pipeline
    param = {
        'classifier__n_estimators': trial.suggest_int('classifier__n_estimators', 200, 2000, step=100),
        'classifier__learning_rate': trial.suggest_float('classifier__learning_rate', 0.01, 0.2, log=True),
        'classifier__max_depth': trial.suggest_int('classifier__max_depth', 3, 12),
        'classifier__subsample': trial.suggest_float('classifier__subsample', 0.5, 1.0),
        'classifier__colsample_bytree': trial.suggest_float('classifier__colsample_bytree', 0.5, 1.0),
        'classifier__gamma': trial.suggest_float('classifier__gamma', 1e-8, 1.0, log=True),
        'classifier__min_child_weight': trial.suggest_int('classifier__min_child_weight', 1, 15),
        'classifier__reg_alpha': trial.suggest_float('classifier__reg_alpha', 1e-8, 1.0, log=True),
        'classifier__reg_lambda': trial.suggest_float('classifier__reg_lambda', 1e-8, 1.0, log=True),
        'classifier__scale_pos_weight': trial.suggest_float('classifier__scale_pos_weight', 1.0, 4.0)
    }


In [36]:
    # Apply the selected parameters to the model pipeline
    model.set_params(**param)


NameError: name 'param' is not defined

In [37]:
  # Perform stratified k-fold cross-validation to get a robust score
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = []
    
    for train_index, val_index in kf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        f1_scores.append(f1_score(y_val, preds, average='binary'))

    return np.mean(f1_scores)

IndentationError: unexpected indent (788712493.py, line 2)

In [ ]:
# Create and run the Optuna study
print("\nSTEP 5: Starting hyperparameter optimization with Optuna... 🚀")
study = optuna.create_study(direction='maximize', study_name='churn_optimization')
study.optimize(objective, n_trials=100, timeout=1800) # Runs for 100 trials or max 30 minutes

In [ ]:

# ------------------------------------------------------------------
# 6. TRAIN FINAL MODEL AND GENERATE SUBMISSION
# ------------------------------------------------------------------

print("\n✅ Optimization finished!")
print(f"Best F1 Score from CV: {study.best_value:.4f}")
print("Best hyperparameters found:")
best_params_display = {k.split('__')[1]: v for k, v in study.best_params.items()}
for key, value in best_params_display.items():
    print(f"  - {key}: {value}")

In [ ]:
# Extract the best hyperparameters in the correct format for the classifier
best_params_final = {key.split('__')[1]: value for key, value in study.best_params.items()}
best_params_final.update({
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'use_label_encoder': False,
    'random_state': 42,
    'n_jobs': -1
})

In [ ]:
# Initialize and train the final model using the best parameters on the *entire* training dataset
final_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(**best_params_final))
])

In [ ]:
print("\nSTEP 6: Training final model and creating submission file...")
final_model.fit(X, y)

In [ ]:
# Make predictions on the test set
test_predictions_encoded = final_model.predict(test_df)

In [ ]:
# Create and save the submission file
submission_df = pd.DataFrame({'id': test_ids, 'Churn': test_predictions})
submission_df.to_csv('submission.csv', index=False)